In [8]:
import time
import theano
import lasagne
import numpy as np

import theano.tensor as T

from utils import custom_sgd, iterate_minibatches
from kron_layer import KronLayer, SimpleKronLayer
from uv_kron_layer import UVKronLayer
from lowrank_layer import LowRankLayer, SimpleLowRankLayer


def build_custom_mlp(input_var=None, widths=None, drop_input=.2,
                     drop_hidden=.5, type="dense", rank=2):
    # By default, this creates the same network as `build_mlp`, but it can be
    # customized with respect to the number and size of hidden layers. This
    # mostly showcases how creating a network in Python code can be a lot more
    # flexible than a configuration file. Note that to make the code easier,
    # all the layers are just called `network` -- there is no need to give them
    # different names if all we return is the last one we created anyway; we
    # just used different names above for clarity.

    widths = widths if widths is not None else [100, 100]
    manifolds = {}

    # Input layer and dropout (with shortcut `dropout` for `DropoutLayer`):
    network = lasagne.layers.InputLayer(shape=(None, 1, 28, 28),
                                        input_var=input_var)
    if drop_input:
        network = lasagne.layers.dropout(network, p=drop_input)
    # Hidden layers and dropout:
    nonlin = lasagne.nonlinearities.rectify

    # Convolutional layer with 32 kernels of size 5x5. Strided and padded
    # convolutions are supported as well; see the docstring.
    network = lasagne.layers.Conv2DLayer(
            network, num_filters=32, filter_size=(5, 5),
            stride=1, pad=2,
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotUniform())
    # Expert note: Lasagne provides alternative convolutional layers that
    # override Theano's choice of which implementation to use; for details
    # please see http://lasagne.readthedocs.org/en/latest/user/tutorial.html.
    network = lasagne.layers.MaxPool2DLayer(network, pool_size=(2, 2))

    # Another convolution with 32 5x5 kernels, and another 2x2 pooling:
    network = lasagne.layers.Conv2DLayer(
            network, num_filters=32, filter_size=(5, 5),
            stride=1, pad=2,
            nonlinearity=lasagne.nonlinearities.rectify)
    network = lasagne.layers.MaxPool2DLayer(network, pool_size=(2, 2))

    if type == "dense":
        network = lasagne.layers.DenseLayer(
            network, widths[0], nonlinearity=nonlin)
    else:
        if type == "lowrank":
            network = LowRankLayer(network,
                                   widths[0],
                                   rank=rank,
                                   name="rieman_low_rank_0")
            manifolds["rieman_low_rank_0"] = network.manifold
        elif type == "slowrank":
            network = LowRankLayer(network,
                                   widths[0],
                                   rank=rank,
                                   name="simple_low_rank_0")
        else:
            raise ValueError("type must be one of 2 variants: lowrank' or 'slowrank', but is '{}'".format(type))
    for width in widths[1:]:
        network = lasagne.layers.DenseLayer(
                network, width, nonlinearity=nonlin)
        if drop_hidden:
            network = lasagne.layers.dropout(network, p=drop_hidden)
    # Output layer:
    softmax = lasagne.nonlinearities.softmax
    network = lasagne.layers.DenseLayer(network, 10, nonlinearity=softmax)
    return network, manifolds


def generate_train_acc(input_X=None, target_y=None, widths=None, type="dense", lr=1e-2, rank=2):
    input_X = T.tensor4("X") if input_X is None else input_X
    target_y = T.vector("target Y integer", dtype='int32') if target_y is None else target_y
    widths = [100] if widths is None else widths
    dense_output, manifolds = build_custom_mlp(input_X, widths=widths, type=type, rank=rank)

    y_predicted = lasagne.layers.get_output(dense_output)


    all_weights = lasagne.layers.get_all_params(dense_output)


    loss = lasagne.objectives.categorical_crossentropy(y_predicted,target_y).mean()
    accuracy = lasagne.objectives.categorical_accuracy(y_predicted,target_y).mean()

    updates_sgd = custom_sgd(loss, all_weights, learning_rate=lr, manifolds=manifolds)


    train_fun = theano.function([input_X,target_y],[loss, accuracy],updates=updates_sgd)
    accuracy_fun = theano.function([input_X,target_y],accuracy)
    return train_fun, accuracy_fun


def comparison(X_train,y_train,X_val,y_val,X_test,y_test, kron_params=None):
    import pickle
    n_trials = 5
    kron_params = np.ones(n_trials, dtype=int) * 10 if kron_params is None else kron_params
    lr = 0.0316227766017
    num_epochs = 60

    batch_size = 100

    hidden_units = [256]

    trains, accs = generate_train_acc(widths=hidden_units, type="dense", lr=lr)
    trains, accs = list(zip(*([(trains, accs)]
                              + [generate_train_acc(widths=hidden_units, type="lowrank", lr=lr, rank=kron_param) for kron_param in kron_params]
                              + [generate_train_acc(widths=hidden_units, type="slowrank", lr=lr,rank=kron_param) for kron_param in kron_params])))

    names = ["dense"] + ["rieman_lowrank({})".format(i) for i in range(n_trials)]\
                      + ["simple_lowrank({})".format(i) for i in range(n_trials)]
    results = {}

    for train, acc, name in zip(trains, accs, names):
        res = {}
        res["train_fun"] = train
        res["accuracy_fun"] = acc
        res["train_err"] = []
        res["train_acc"] = []
        res["epoch_times"] = []
        res["val_acc"] = []
        results[name] = res

    for epoch in range(num_epochs):
        for (res_name, res) in results.items():
            train_err = 0
            train_acc = 0
            train_batches = 0
            start_time = time.time()
            for batch in iterate_minibatches(X_train, y_train,batch_size):
                inputs, targets = batch
                train_err_batch, train_acc_batch= res["train_fun"](inputs, targets)
                train_err += train_err_batch
                train_acc += train_acc_batch
                train_batches += 1

            # And a full pass over the validation data:
            val_acc = 0
            val_batches = 0
            for batch in iterate_minibatches(X_val, y_val, batch_size):
                inputs, targets = batch
                val_acc += res["accuracy_fun"](inputs, targets)
                val_batches += 1

            # Then we print the results for this epoch:
            print("for {}".format(res_name))
            print("Epoch {} of {} took {:.3f}s".format(
                epoch + 1, num_epochs, time.time() - start_time))

            print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))
            print("  train accuracy:\t\t{:.2f} %".format(
                train_acc / train_batches * 100))
            print("  validation accuracy:\t\t{:.2f} %".format(
                val_acc / val_batches * 100))
            res["train_err"].append(train_err / train_batches)
            res["train_acc"].append(train_acc / train_batches * 100)
            res["val_acc"].append(val_acc / val_batches * 100)
        temp = {key: [results[key]['train_fun'], results[key]['accuracy_fun']] for key in results}
        for res in results.values():
            res.pop('train_fun')
            res.pop('accuracy_fun')
        
        with open("compare_lowrank_10.dict", 'wb') as pickle_file:
            pickle.dump(results, pickle_file)
        for key in results:
            results[key]['train_fun'] = temp[key][0]
            results[key]['accuracy_fun'] = temp[key][1]

def determine(X_train,y_train,X_val,y_val,X_test,y_test, type="lowrank", lrates=None):
    lrates = np.logspace(-3, -1, 5) if lrates is None else lrates
    rank = 10
    import pickle
    num_epochs = 25

    batch_size = 100

    hidden_units = [256]

    trains, accs = list(zip(*([generate_train_acc(widths=hidden_units, type=type, rank=rank) for lr in lrates])))

    names = ["{}({})".format(type, lr) for lr in lrates]
    results = {}

    for train, acc, name in zip(trains, accs, names):
        res = {}
        res["train_fun"] = train
        res["accuracy_fun"] = acc
        res["train_err"] = []
        res["train_acc"] = []
        res["epoch_times"] = []
        res["val_acc"] = []
        results[name] = res

    for epoch in range(num_epochs):
        for (res_name, res) in results.items():
            train_err = 0
            train_acc = 0
            train_batches = 0
            start_time = time.time()
            for batch in iterate_minibatches(X_train, y_train,batch_size):
                inputs, targets = batch
                train_err_batch, train_acc_batch= res["train_fun"](inputs, targets)
                train_err += train_err_batch
                train_acc += train_acc_batch
                train_batches += 1

            # And a full pass over the validation data:
            val_acc = 0
            val_batches = 0
            for batch in iterate_minibatches(X_val, y_val, batch_size):
                inputs, targets = batch
                val_acc += res["accuracy_fun"](inputs, targets)
                val_batches += 1

            # Then we print the results for this epoch:
            print("for {}".format(res_name))
            print("Epoch {} of {} took {:.3f}s".format(
                epoch + 1, num_epochs, time.time() - start_time))

            print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))
            print("  train accuracy:\t\t{:.2f} %".format(
                train_acc / train_batches * 100))
            print("  validation accuracy:\t\t{:.2f} %".format(
                val_acc / val_batches * 100))
            res["train_err"].append(train_err / train_batches)
            res["train_acc"].append(train_acc / train_batches * 100)
            res["val_acc"].append(val_acc / val_batches * 100)
        temp = {key: [results[key]['train_fun'], results[key]['accuracy_fun']] for key in results}
        for res in results.values():
            res.pop('train_fun')
            res.pop('accuracy_fun')
        
        with open("determine_history_{}.dict".format(type), 'wb') as pickle_file:
            pickle.dump(results, pickle_file)
        for key in results:
            results[key]['train_fun'] = temp[key][0]
            results[key]['accuracy_fun'] = temp[key][1]

In [7]:
if __name__ == "__main__":
    from mnist import load_dataset
    X_train,y_train,X_val,y_val,X_test,y_test = load_dataset()
    print(X_train.shape,y_train.shape)

    determine(X_train,y_train,X_val,y_val,X_test,y_test, type="lowrank")

((50000, 1, 28, 28), (50000,))
for lowrank(0.1)
Epoch 1 of 25 took 32.964s
  training loss (in-iteration):		1.280312
  train accuracy:		58.10 %
  validation accuracy:		85.97 %
for lowrank(0.001)
Epoch 1 of 25 took 32.857s
  training loss (in-iteration):		1.395296
  train accuracy:		56.31 %
  validation accuracy:		85.99 %
for lowrank(0.0316227766017)
Epoch 1 of 25 took 32.853s
  training loss (in-iteration):		1.535191
  train accuracy:		49.08 %
  validation accuracy:		83.94 %
for lowrank(0.01)
Epoch 1 of 25 took 32.881s
  training loss (in-iteration):		1.387994
  train accuracy:		53.54 %
  validation accuracy:		83.77 %
for lowrank(0.00316227766017)
Epoch 1 of 25 took 32.877s
  training loss (in-iteration):		1.494428
  train accuracy:		51.91 %
  validation accuracy:		84.15 %
for lowrank(0.1)
Epoch 2 of 25 took 32.844s
  training loss (in-iteration):		0.398253
  train accuracy:		87.76 %
  validation accuracy:		91.46 %
for lowrank(0.001)
Epoch 2 of 25 took 32.866s
  training loss (in-itera